In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import zipfile
from xgboost import XGBRegressor
import os
from glob import glob
import matplotlib.pyplot as plt
from skimage import data
from scipy.stats import gmean

In [ ]:
# Paths
gt_path = '../input/esahyber/train_data/train_data/train_gt.csv'
wavelength_path = '../input/esahyber/train_data/train_data/wavelengths.csv'
hsi_path = '../input/esahyber/train_data/train_data/train_data/1000.npz'

tr_path = "../input/esahyber/train_data/train_data/train_data"
te_path = "../input/esahyber/test_data/test_data"

In [ ]:

gt_df = pd.read_csv(gt_path)
wavelength_df = pd.read_csv(wavelength_path)

## Displaying one hyperspectral band

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 5))
band_id = 100
wavelength = wavelength_df.loc[band_id-1]

with np.load(hsi_path) as npz:
    arr = np.ma.MaskedArray(**npz)

axs[0].imshow(arr[band_id,:,:].data)
axs[1].imshow(arr[band_id,:,:])
plt.suptitle(f'Representation of band {int(wavelength["band_no"])} ({wavelength["wavelength"]} nm)', fontsize=15)
plt.show()

## Displaying the aggregated spectral curve for a field

In [ ]:
fig = plt.figure(figsize=(10, 5))

masked_scene_mean_spectral_reflectance = [arr[i,:,:].mean() for i in range(arr.shape[0])]
full_scene_mean_spectral_reflectance = [arr[i,:,:].data.mean() for i in range(arr.shape[0])]

plt.plot(wavelength_df['wavelength'], full_scene_mean_spectral_reflectance, label='Full image')
plt.plot(wavelength_df['wavelength'], masked_scene_mean_spectral_reflectance, label='Masked image')

plt.xlabel('[nm]')
plt.legend()
plt.title(f'Average reflectance ({hsi_path.split("/")[-1]})')
plt.show()

## Load the data

In [ ]:
class SpectralCurveFiltering():
    """
    Create a histogram (a spectral curve) of a 3D cube, using the merge_function
    to aggregate all pixels within one band. The return array will have
    the shape of [CHANNELS_COUNT]
    """

    def __init__(self, merge_function = np.mean):
        self.merge_function = merge_function

    def __call__(self, sample: np.ndarray):
        return self.merge_function(sample.data, axis=(1, 2))

In [ ]:
def load_data(directory: str, merge_functions, wv, prefixes):
    """Load each cube, reduce its dimensionality and append to array.

    Args:
        directory (str): Directory to either train or test set
    Returns:
        [type]: A list with spectral curve for each sample.
    """
    hsi_data = None
    for idx, merge_function in enumerate(merge_functions):
      data = []
      filtering = SpectralCurveFiltering(merge_function = merge_function)
      all_files = np.array(
          sorted(
              glob(os.path.join(directory, "*.npz")),
              key=lambda x: int(os.path.basename(x).replace(".npz", "")),
          )
      )
      for file_name in all_files:
          with np.load(file_name) as npz:
              arr = np.ma.MaskedArray(**npz)
                
          # Standard Normal Variate (SNV)
          #arr = NormBand(arr)
          #arr1 = arr.data * 10e-5
          #arr = np.ma.masked_array(arr1,arr.mask)
          arr = filtering(arr)
          data.append(arr)

      data = np.array(data)
      #data = pd.DataFrame(data,columns=[wv], dtype=np.float64)
      data = pd.DataFrame(data,columns=wv, dtype=np.float64)
      data = data.add_prefix(f'{prefixes[idx]}_wv_')
      if hsi_data is None:
        hsi_data = data #pd.DataFrame(data)
      else:
        hsi_data = pd.concat([hsi_data, data], axis=1)

    return hsi_data


def load_gt(file_path: str):
    """Load labels for train set from the ground truth file.
    Args:
        file_path (str): Path to the ground truth .csv file.
    Returns:
        [type]: 2D numpy array with soil properties levels
    """
    gt_file = pd.read_csv(file_path)
    labels = gt_file[["P", "K", "Mg", "pH"]].values
    return labels



In [ ]:
prefixes = ['mean']#, 'std'] # , 'min', 'max']
merge_functions = [np.mean]#, np.std] #, np.min, np.max]
wv = (list(wavelength_df['wavelength']))
X_tr = load_data(tr_path, merge_functions, wv, prefixes)

In [ ]:
X_te = load_data(te_path, merge_functions, wv, prefixes)

In [ ]:
y = load_gt(gt_path)

y = pd.DataFrame(y, columns=["P", "K", "Mg", "pH"])

In [ ]:
X_tr.head(3)

In [ ]:
prefixes = ['std', 'min', 'max', 'median']
merge_functions = [np.std, np.min, np.max, np.median]
wv = (list(wavelength_df['wavelength']))
X_tr_stats = load_data(tr_path, merge_functions, wv, prefixes)

X_te_stats = load_data(te_path, merge_functions, wv, prefixes)

### Narrow bands Indices

In [ ]:
# Indices of important features
def NBI(df, features):
  X_indices = None
  names = []
  for col1 in features:
    indices = []
    for col2 in features:
      if col1 !=col2:
        name = f'Index_{col1}_{col2}'
        #name2 = f'{col1}_multi_{col2}'
        #name3 =  f'{col1}_div_{col2}'

        index = (np.array(df[col2]) - np.array(df[col1])) / (np.array(df[col2]) + np.array(df[col1]))
        names.append(name)

        indices.append(pd.DataFrame(index, columns=[name]))

        #multi = np.array(df[col2]) * np.array(df[col1])
        #names.append(name2)
        #indices.append(pd.DataFrame(multi, columns=[name2]))


        #divi = np.array(df[col2]) / np.array(df[col1])
        #names.append(name3)
        #indices.append(pd.DataFrame(divi, columns=[name3]))

    X_new_indices = pd.concat(indices, axis=1)

    if X_indices is None:
      X_indices = X_new_indices
    else:
      X_indices = pd.concat([X_indices, X_new_indices] , axis=1, ignore_index=False)

  return  X_indices

In [ ]:
# All possible band combinations
features = X_tr.columns

X_tr_indices = NBI(X_tr, features)
X_te_indices = NBI(X_te, features)

In [ ]:
X_tr = pd.concat([X_tr, X_tr_indices] , axis=1, ignore_index=False)
X_te = pd.concat([X_te, X_te_indices] , axis=1, ignore_index=False)

print(X_tr.shape)
print(X_te.shape)

In [ ]:
X_tr = pd.concat([X_tr, X_tr_stats] , axis=1, ignore_index=False)
X_te = pd.concat([X_te, X_te_stats] , axis=1, ignore_index=False)

print(X_tr.shape)
print(X_te.shape)

### Feature Selection

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
import pickle

In [ ]:
#with open('../input/best-var-lists/P_selected_features.pkl', 'rb') as f:
#    P_selected_features = pickle.load(f)
    
#with open('../input/best-var-lists/K_selected_features.pkl', 'rb') as f:
 #   K_selected_features = pickle.load(f)
    
#with open('../input/best-var-lists/Mg_selected_features.pkl', 'rb') as f:
 #   Mg_selected_features = pickle.load(f)
    
#with open('../input/best-var-lists/pH_selected_features.pkl', 'rb') as f:
  #  pH_selected_features = pickle.load(f)
    

In [ ]:
rfe = RFE(estimator=RandomForestRegressor(n_estimators=100,random_state=2022), n_features_to_select=500, step=0.40)
model = RandomForestRegressor(n_estimators=100, random_state=2022)
pipeline = Pipeline(steps=[('s',rfe),('m',model)])

pipeline.fit(X_tr, y['P'])

cols = list(X_tr.columns)
P_selected_features = []
for idx, col in enumerate(cols):
    if rfe.support_[idx]:
        P_selected_features.append(col)
        

with open('P_selected_features.pkl', 'wb') as f:
    pickle.dump(P_selected_features, f)
    
    

In [ ]:
rfe = RFE(estimator=RandomForestRegressor(n_estimators=100,random_state=2022), n_features_to_select=500, step=0.40)
model = RandomForestRegressor(n_estimators=100, random_state=2022)
pipeline = Pipeline(steps=[('s',rfe),('m',model)])

pipeline.fit(X_tr, y['K'])

cols = list(X_tr.columns)
K_selected_features = []
for idx, col in enumerate(cols):
    if rfe.support_[idx]:
        K_selected_features.append(col)
        
with open('K_selected_features.pkl', 'wb') as f:
    pickle.dump(K_selected_features, f)
    

In [ ]:
rfe = RFE(estimator=RandomForestRegressor(n_estimators=100,random_state=2022), n_features_to_select=500, step=0.40)
model = RandomForestRegressor(n_estimators=100, random_state=2022)
pipeline = Pipeline(steps=[('s',rfe),('m',model)])

pipeline.fit(X_tr, y['Mg'])

cols = list(X_tr.columns)
Mg_selected_features = []
for idx, col in enumerate(cols):
    if rfe.support_[idx]:
        Mg_selected_features.append(col)
        
with open('Mg_selected_features.pkl', 'wb') as f:
    pickle.dump(Mg_selected_features, f)

In [ ]:
rfe = RFE(estimator=RandomForestRegressor(n_estimators=100,random_state=2022), n_features_to_select=500, step=0.40)
model = RandomForestRegressor(n_estimators=100, random_state=2022)
pipeline = Pipeline(steps=[('s',rfe),('m',model)])

pipeline.fit(X_tr, y['pH'])

cols = list(X_tr.columns)
pH_selected_features = []
for idx, col in enumerate(cols):
    if rfe.support_[idx]:
        pH_selected_features.append(col)
        
        
with open('pH_selected_features.pkl', 'wb') as f:
    pickle.dump(pH_selected_features, f)

In [ ]:
P_X_tr = X_tr[P_selected_features]
P_X_te = X_te[P_selected_features]

K_X_tr = X_tr[K_selected_features]
K_X_te = X_te[K_selected_features]

Mg_X_tr = X_tr[Mg_selected_features]
Mg_X_te = X_te[Mg_selected_features]

pH_X_tr = X_tr[pH_selected_features]
pH_X_te = X_te[pH_selected_features]

### Interactions

In [ ]:
# Indices of important features
def interactions(df, features):
    X_interact = None
    names = []
    p_columns = []
    for col1 in features:
        interact = []
        p_columns.append(col1)
       # print(p_columns)

        for col2 in features:
            if col2 not in p_columns:
                #print(col2)
                
                name = f'{col1}_multi_{col2}'
                name2 =  f'{col1}_div_{col2}'
                multi = np.array(df[col2]) * np.array(df[col1])
                names.append(name)
                interact.append(pd.DataFrame(multi, columns=[name]))


                divi = np.array(df[col2]) / np.array(df[col1] + 1e-6)
                names.append(name2)
                interact.append(pd.DataFrame(divi, columns=[name2]))
        #print('-----------------')
        if len(p_columns) != len(features):
            X_new_interact = pd.concat(interact, axis=1)

            if X_interact is None:
                X_interact = X_new_interact
            else:
                X_interact = pd.concat([X_interact, X_new_interact] , axis=1, ignore_index=False)

    return  X_interact

In [ ]:
# All possible band combinations
features = P_X_tr.columns[:10]

X_tr_intrs = interactions(P_X_tr[features], features)
X_te_intrs = interactions(P_X_te[features], features)

P_X_tr = pd.concat([P_X_tr, X_tr_intrs] , axis=1, ignore_index=False)
P_X_te = pd.concat([P_X_te, X_te_intrs] , axis=1, ignore_index=False)

print(P_X_tr.shape)
print(P_X_te.shape)

In [ ]:
# All possible band combinations
features = K_X_tr.columns[:10]

X_tr_intrs = interactions(K_X_tr[features], features)
X_te_intrs = interactions(K_X_te[features], features)

K_X_tr = pd.concat([K_X_tr, X_tr_intrs] , axis=1, ignore_index=False)
K_X_te = pd.concat([K_X_te, X_te_intrs] , axis=1, ignore_index=False)

print(K_X_tr.shape)
print(K_X_te.shape)

In [ ]:
# All possible band combinations
features = Mg_X_tr.columns[:10]

X_tr_intrs = interactions(Mg_X_tr[features], features)
X_te_intrs = interactions(Mg_X_te[features], features)

Mg_X_tr = pd.concat([Mg_X_tr, X_tr_intrs] , axis=1, ignore_index=False)
Mg_X_te = pd.concat([Mg_X_te, X_te_intrs] , axis=1, ignore_index=False)

print(Mg_X_tr.shape)
print(Mg_X_te.shape)

In [ ]:
# All possible band combinations
features = pH_X_tr.columns[:10]

X_tr_intrs = interactions(pH_X_tr[features], features)
X_te_intrs = interactions(pH_X_te[features], features)

pH_X_tr = pd.concat([pH_X_tr, X_tr_intrs] , axis=1, ignore_index=False)
pH_X_te = pd.concat([pH_X_te, X_te_intrs] , axis=1, ignore_index=False)

print(pH_X_tr.shape)
print(pH_X_te.shape)

### Generating baseline solution

In [ ]:
from sklearn.model_selection import StratifiedKFold, StratifiedGroupKFold
import math
import warnings
warnings.filterwarnings("ignore")

In [ ]:
P_data = pd.concat([P_X_tr, y['P']], axis=1).reset_index(drop=True)
K_data = pd.concat([K_X_tr, y['K']], axis=1).reset_index(drop=True)
Mg_data = pd.concat([Mg_X_tr, y['Mg']], axis=1).reset_index(drop=True)
pH_data = pd.concat([pH_X_tr, y['pH']], axis=1).reset_index(drop=True)

In [ ]:
def skf_split(data_df, target):
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=2022)
    num_splits = 5
    num_bins = math.floor(len(data_df) / num_splits)  # num of bins to be created
    bins_on = data_df[target]  # variable to be used for stratification
    qc = pd.cut(bins_on.tolist(), num_bins)  # divides data in bins
    data_df['bins'] = qc.codes
    
    val_index = []
    X_train, X_val = [], []
    Y_train, Y_val = [], []
    for idx , (train, val) in enumerate(skf.split(data_df,data_df['bins'])):
        x_train, x_val, y_train, y_val = data_df.iloc[train], data_df.iloc[val], data_df[target].iloc[train], data_df[target].iloc[val]
        x_train = x_train.drop(['bins', target], axis=1)
        x_val = x_val.drop(['bins', target], axis=1)
        
        val_index.append(val)
        X_train.append(x_train)
        Y_train.append(y_train)
        
        X_val.append(x_val)
        Y_val.append(y_val)
    
    return X_train, X_val, Y_train, Y_val, val_index

In [ ]:
X_P_tr,X_P_val, y_P_tr, y_P_val, P_val_index = skf_split(P_data, 'P')
X_K_tr,X_K_val, y_K_tr, y_K_val, K_val_index = skf_split(K_data, 'K')
X_Mg_tr,X_Mg_val, y_Mg_tr, y_Mg_val, Mg_val_index = skf_split(Mg_data, 'Mg')
X_pH_tr,X_pH_val, y_pH_tr, y_pH_val, pH_val_index = skf_split(pH_data, 'pH')

In [ ]:
#from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
import random

In [ ]:
# Evaluation metric
class BaselineRegressor:
    """
    Baseline regressor, which calculates the mean value of the target from the training
    data and returns it for each testing sample.
    """
    def __init__(self):
        self.mean = 0

    def fit(self, X_train: np.ndarray, y_train: np.ndarray):
      self.mean = np.mean(y_train, axis=0)
      #self.classes_count = y_train.shape[1]
      self.classes_count = 1
      return self

    def predict(self, X_test: np.ndarray):
      return np.full((len(X_test), self.classes_count), self.mean)


In [ ]:
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import ExtraTreesRegressor

In [ ]:
def EvaluationMetric(baseline_reg, x_val, y_val, rf_val_preds, cat_val_preds, lgbm_val_preds, xgbm_val_preds, et_val_preds,
                     rf_te_preds, cat_te_preds, lgbm_te_preds, xgbm_te_preds, et_te_preds):
    #baseline_model = baseline_reg
    baseline_predictions = baseline_reg.predict(x_val)
    baseline_predictions = baseline_predictions.squeeze()
    baselines = np.mean((y_val - baseline_predictions) ** 2, axis=0)
    
    # Calculate MSE
    best_score = np.inf
    for i in range(5000):
        weights = np.random.dirichlet(np.ones(5),size=1)
        a = weights[0][0]
        b = weights[0][1]
        c = weights[0][2]
        d = weights[0][3]
        e = weights[0][4]
        
        val_preds = (a * rf_val_preds) + (b * cat_val_preds) + (c * lgbm_val_preds) + (d * xgbm_val_preds) + (e * et_val_preds)
        mse = np.mean((y_val - val_preds) ** 2, axis=0)
        score = mse / baselines
        
        if score < best_score:
            #print(score)
            best_score = score
            best_a = a
            best_b = b
            best_c = c
            best_d = d
            best_e = e
    val_preds = (best_a * rf_val_preds) + (best_b * cat_val_preds) + (best_c * lgbm_val_preds) + (best_d * xgbm_val_preds) + (best_e * et_val_preds)
    te_preds = (best_a * rf_te_preds) + (best_b * cat_te_preds) + (best_c * lgbm_te_preds) + (best_d * xgbm_te_preds) + (best_e * et_te_preds)
    
    
    # Calculate the final score
    #final_score = np.mean(scores)

    return best_score, te_preds, val_preds

In [ ]:
def regressor(x_train, y_train, x_val, y_val, X_test):
    rf_regressor = RandomForestRegressor(random_state=2022, criterion='squared_error', n_estimators=700)
    rf_regressor.fit(x_train,y_train)
    rf_val_preds =  rf_regressor.predict(x_val)
    rf_te_preds = rf_regressor.predict(X_test)

    
    cat_regressor = CatBoostRegressor(n_estimators=3000,learning_rate=0.01,logging_level='Silent',random_state=2022, early_stopping_rounds=300)
    cat_regressor.fit(x_train,y_train)
    cat_val_preds =  cat_regressor.predict(x_val)
    cat_te_preds = cat_regressor.predict(X_test)

    
    
    lgbm_regressor = LGBMRegressor(learning_rate=0.01, n_estimators=3000,deterministic=True,random_state=2022, n_jobs=- 1) #,subsample=0.65,subsample_freq=20, colsample_bytree=0.65,
    lgbm_regressor.fit(x_train,y_train)
    lgbm_val_preds =  lgbm_regressor.predict(x_val)
    lgbm_te_preds = lgbm_regressor.predict(X_test)

    
    xgbm_regressor =XGBRegressor(n_estimators = 2000,learning_rate = 0.01,seed=2022,random_state = 2022)
    xgbm_regressor.fit(x_train,y_train)
    xgbm_val_preds =  xgbm_regressor.predict(x_val)
    xgbm_te_preds = xgbm_regressor.predict(X_test)

    
    et_regressor = ExtraTreesRegressor(n_estimators=700,criterion = "squared_error",
                                       random_state=2022) # bootstrap = True, max_samples = 0.85, warm_start=True,
    et_regressor.fit(x_train,y_train)
    et_val_preds =  et_regressor.predict(x_val)
    et_te_preds = et_regressor.predict(X_test)

    

    # Predictions
    baseline_reg = BaselineRegressor()
    baseline_reg = baseline_reg.fit(x_train, y_train)
    
    score, te_preds, train_preds = EvaluationMetric(baseline_reg, x_val, y_val, rf_val_preds, cat_val_preds,lgbm_val_preds, xgbm_val_preds,et_val_preds,
                                       rf_te_preds, cat_te_preds, lgbm_te_preds, xgbm_te_preds, et_te_preds)

    # train predictions
    #train_preds = regressor.predict(x_val)

    # Test predictions
    #preds = regressor.predict(X_test)

    return score, te_preds , train_preds

In [ ]:
np.random.seed(2022)

#kf = KFold(n_splits =5,shuffle=True,random_state=2022)

final_scores = []
#val_predictions = []
final_predictions = []
train_preds = np.zeros((X_tr.shape[0],4)) 

for i, s in enumerate(range(5)):
    print(f'######### FOLD {i+1} / 5')
    scores = []
    preds = np.zeros((X_te.shape[0],4))

    # Predict P 
    x_train, x_val, y_train,  y_val = X_P_tr[s], X_P_val[s], y_P_tr[s], y_P_val[s]
    score, P_preds , P_train_preds = regressor(x_train, y_train, x_val, y_val, P_X_te)
    scores.append(score)
    print(f'P Score: {score}')
    train_preds[P_val_index[s],0] = P_train_preds
    preds[:,0] = P_preds
    

    # Predict k
    x_train, x_val, y_train,  y_val = X_K_tr[s], X_K_val[s], y_K_tr[s], y_K_val[s]
    score,  k_preds, k_train_preds = regressor(x_train, y_train, x_val, y_val, K_X_te)
    scores.append(score)
    print(f'K Score: {score}')
    train_preds[K_val_index[s],1] = k_train_preds
    preds[:,1] =k_preds
    

    # Predict Mg
    x_train, x_val, y_train,  y_val = X_Mg_tr[s], X_Mg_val[s], y_Mg_tr[s], y_Mg_val[s]
    score, Mg_preds, Mg_train_preds = regressor(x_train, y_train, x_val, y_val, Mg_X_te)
    scores.append(score)
    print(f'Mg Score: {score}')
    train_preds[Mg_val_index[s],2] = Mg_train_preds
    preds[:,2] = Mg_preds


    # Predict pH
    x_train, x_val, y_train,  y_val = X_pH_tr[s], X_pH_val[s], y_pH_tr[s], y_pH_val[s]
    score, pH_preds, pH_train_preds = regressor(x_train, y_train, x_val, y_val, pH_X_te)
    scores.append(score)
    print(f'pH Score: {score}')
    train_preds[pH_val_index[s],3] = pH_train_preds
    preds[:,3] = pH_preds

    final_score = np.mean(scores)
    print(f'Overall score: {final_score} ')

    final_scores.append(final_score)

    final_predictions.append(preds)


print('mean scores: {} '.format(np.mean(final_scores)))

In [ ]:
f_train_preds = pd.DataFrame(data = train_preds, columns=["P", "K", "Mg", "pH"])

f_train_preds.to_csv("./train_preds.csv", index_label="sample_index")

In [ ]:
final_test_predictions = gmean(final_predictions, axis=0)

submission = pd.DataFrame(data = final_test_predictions, columns=["P", "K", "Mg", "pH"])
submission.to_csv("./submissions_gmean_cat_rf_lgbm_xgbm_et.csv", index_label="sample_index")